In [2]:
from os import listdir
import os
import keras
from os.path import isfile, join
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,Reshape,LSTM,Conv1D,MaxPooling1D,SimpleRNN
import pandas as pd
#data_class=np.loadtxt(r"C:\Users\USER\Desktop\labels.txt",delimiter=',')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.utils import np_utils   
from sklearn.metrics import f1_score
import logging
from keras.regularizers import l2
from keras import backend as K
from sklearn.svm import SVC 
#K.set_image_dim_ordering('th')
from keras.models import Model
from sklearn import tree
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.naive_bayes import GaussianNB
#from hmmlearn import hmm
print(5)

5


In [3]:
def _extract_segments(clip, filename):
    
    
    FRAMES_PER_SEGMENT = 49  # 41 frames ~= 950 ms segment length @ 22050 Hz
    WINDOW_SIZE = 512 * 49   # 23 ms per frame @ 22050 Hz
    STEP_SIZE = 512 * 49 // 2
    BANDS = 60
    #sum=np.zeros(60,41)
    
    s = 0
    segments1 = []
    #segments2=[]
    #logspec=[]
    #logspec=np.array(logspec)

    normalization_factor = 1 / np.max(np.abs(clip)) 
    clip = clip * normalization_factor

    while len(clip[s * STEP_SIZE:s * STEP_SIZE + WINDOW_SIZE]) == WINDOW_SIZE:
        signal = clip[s * STEP_SIZE:s * STEP_SIZE + WINDOW_SIZE]

        melspec = librosa.feature.mfcc(signal, sr=22050, n_fft=1024, hop_length=512, n_mfcc=BANDS)
        #logspec = librosa.logamplitude(melspec)

        #delta=librosa.feature.delta(melspec,mode='nearest')
        
        segments1.append(melspec)
        #segments2.append(delta)
        s = s + 1
    #for i in range(s-1):
        #sum=sum+segments(i,:,:)
    #segments=sum/s-1
    segments1=np.mean(segments1,axis=0) 
    #segments2=np.mean(segments2,axis=0)

    #segments = pd.concat(segments, ignore_index=True)
    #libc.malloc_trim(0)
    return segments1
print(5)

5


In [4]:

dataclass1=[]
dataclass2=[]
dataclass3=[]
dataclass4=[]
fold1=[]
fold2=[]
fold3=[]
fold4=[]
def extract_features1(dataset_path):
    onlyfiles = [f for f in listdir(dataset_path)]
    for j in range(len(onlyfiles)):
        aud_file=[f for f in listdir(os.path.join(dataset_path,onlyfiles[j]))]        
        for i in range(len(aud_file)):
            if(aud_file[i][1]=='1'):
                wav_file=[f for f in listdir(os.path.join(dataset_path,os.path.join(onlyfiles[j],aud_file[i])))]
                for k in range(len(wav_file)):
                    X,sr=librosa.load( os.path.join(dataset_path,os.path.join(onlyfiles[j],os.path.join(aud_file[i],wav_file[k]))))
                    me1=_extract_segments(X,wav_file[k])
                    fold1.append(me1)
                    dataclass1.append(j)
            elif(aud_file[i][1]=='2'):
                wav_file=[f for f in listdir(os.path.join(dataset_path,os.path.join(onlyfiles[j],aud_file[i])))]
                for k in range(len(wav_file)):
                    X,sr=librosa.load( os.path.join(dataset_path,os.path.join(onlyfiles[j],os.path.join(aud_file[i],wav_file[k]))))                    
                    me2=_extract_segments(X,wav_file[k])
                    fold2.append(me2)
                    dataclass2.append(j)
            elif(aud_file[i][1]=='3'):
                wav_file=[f for f in listdir(os.path.join(dataset_path,os.path.join(onlyfiles[j],aud_file[i])))]
                for k in range(len(wav_file)):
                    X,sr=librosa.load( os.path.join(dataset_path,os.path.join(onlyfiles[j],os.path.join(aud_file[i],wav_file[k]))))
                    me3=_extract_segments(X,wav_file[k])
                    fold3.append(me3)
                    dataclass3.append(j)
            elif(aud_file[i][1]=='4'):
                wav_file=[f for f in listdir(os.path.join(dataset_path,os.path.join(onlyfiles[j],aud_file[i])))]
                for k in range(len(wav_file)):
                    X,sr=librosa.load( os.path.join(dataset_path,os.path.join(onlyfiles[j],os.path.join(aud_file[i],wav_file[k]))))
                    me4=_extract_segments(X,wav_file[k])
                    fold4.append(me4)
                    dataclass4.append(j)
   # return  fold1,fold2,fold3,fold4,fold5,dataclass1,dataclass2,dataclass3,dataclasss4,dataclass5

In [5]:
extract_features1(r"C:\internship\sonidosSubir\allSounds\testdataclean")
print((np.array(fold1)).shape)
print((np.array(fold2)).shape)
print((np.array(fold3)).shape)
print((np.array(fold4)).shape)

(70, 60, 50)
(70, 60, 50)
(70, 60, 50)
(70, 60, 50)


In [6]:
train_x=np.vstack([fold1,fold2,fold3])
train_y=dataclass1+dataclass2+dataclass3
print(train_x.shape)

print(len(train_y))

(210, 60, 50)
210


In [7]:
labelencoder_train=LabelEncoder()
labelencoder_train.fit(train_y)
data_cl=labelencoder_train.transform(train_y)
dummy_class_train=np_utils.to_categorical(data_cl)
print (dummy_class_train.shape)


(210, 7)


In [8]:
labelencoder_train=LabelEncoder()
labelencoder_train.fit(dataclass4)
data_cl=labelencoder_train.transform(dataclass4)
dummy_class_test=np_utils.to_categorical(data_cl)
print (dummy_class_test.shape)


(70, 7)


In [48]:
train_xx=np.expand_dims((train_x),axis=3)
print(train_xx.shape)
#train_xx=np.reshape((train_x),(210,60,50,1))
#train_xx.shape

(210, 60, 50, 1)


In [49]:
test_xx=np.expand_dims((fold4),axis=3)
test_xx.shape
test_xx=np.reshape((fold4),(70,60,50,1))
test_xx.shape

(70, 60, 50, 1)

In [50]:
from keras.layers.normalization import BatchNormalization
model=Sequential()
'''
model.add(Conv1D(filters=200, kernel_size=5, input_shape=(3000,1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5 ))
model.add(BatchNormalization())
model.add(Conv1D(filters=200, kernel_size=5))
model.add(MaxPooling1D(pool_size=5 ))
model.add(BatchNormalization())
'''#model.add(Conv1D(filters=10, kernel_size=5))
#model.add(MaxPooling1D(pool_size=5 ))

model.add(Conv2D(100, kernel_size=(5,5),activation='relu',input_shape=(60,50,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))
model.add(BatchNormalization())
model.add(Dropout(.1))
model.add(Conv2D(filters=100,kernel_size=(1,1),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1),strides=(1,3)))
model.add(BatchNormalization())
#model.add(Conv2D(filters=100,kernel_size=(1,1),activation='relu'))
#model.add(MaxPooling2D(pool_size=(1,1),strides=(1,3)))
#model.add(BatchNormalization())
#model.add(Reshape((100, -1)))
#'''
#model.add(SimpleRNN(500,activation='tanh',input_shape=(3000,1)))
#model.add(LSTM(100,activation='tanh',input_shape=(1600,1)))
model.add(Dropout(0.1))
model.add(Reshape((1, -1)))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.1))
#model.add(Dense(num_classes, activation='softmax'))
#model.add(Dense(500, activation='relu'))
#model.add(Dropout(0.1))
#model.add(Dense(500, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.1))
model.add(Dense(500, activation='relu'))
model.add(BatchNormalization())
#clf = hmm.GaussianHMM()
clf = SVC(kernel='poly',degree=4)
#clf1=SVC(kernel='poly',degree=3)
#clf2=SVC(kernel='poly',degree=5)
#clf3=SVC(kernel='poly',degree=2)
#clf1 = tree.DecisionTreeClassifier()
#clf2=RandomForestClassifier()
#clf3=GaussianNB()
#eclf1 = VotingClassifier(estimators=[('svm', clf), ('gnb',clf3)])
#clf=linear_model.LogisticRegression()
#model.add(Dense(7, activation='softmax'))
#model.add(Dropout(0.1))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 56, 46, 100)       2600      
_________________________________________________________________
batch_normalization_15 (Batc (None, 56, 46, 100)       400       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 55, 45, 100)       0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 55, 45, 100)       400       
_________________________________________________________________
dropout_9 (Dropout)          (None, 55, 45, 100)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 55, 45, 100)       10100     
_________________________________________________________________
batch_normalization_17 (Batc (None, 55, 45, 100)       400       
__________

In [52]:
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[13].output])
layer_output = get_3rd_layer_output([train_xx])[0]
layer_output.shape


(210, 500)

In [53]:
layer_output_train=np.reshape((layer_output),(210,500))
layer_output_train.shape

(210, 500)

In [54]:
get_3rd_layer_output1 = K.function([model.layers[0].input],
                                  [model.layers[13].output])
layer_output_1= get_3rd_layer_output1([test_xx])[0]
layer_output_1.shape

(70, 500)

In [55]:
layer_output_test=np.reshape((layer_output_1),(70,500))
layer_output_test.shape


(70, 500)

In [56]:
clf.fit(layer_output_train,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=4, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [57]:
y_pred=clf.predict(layer_output_test)

In [58]:
f1_score(dataclass4, y_pred,average='macro') 

0.66580576849339523

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
#y_pred = svclassifier.predict(test_xx)
#y_pred


In [22]:
model.fit(train_xx, dummy_class_train, batch_size=1, epochs=5)
model.summary()

Epoch 1/5
210/210 [==============================] - 37s 176ms/step - loss: 1.8245 - acc: 0.2238
Epoch 2/5
210/210 [==============================] - 38s 179ms/step - loss: 1.7941 - acc: 0.2524
Epoch 3/5
210/210 [==============================] - 38s 179ms/step - loss: 1.8111 - acc: 0.2333
Epoch 4/5
210/210 [==============================] - 38s 182ms/step - loss: 1.8027 - acc: 0.2429
Epoch 5/5
210/210 [==============================] - 38s 180ms/step - loss: 1.8224 - acc: 0.2143
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
reshape_2 (Reshape)          (None, 1, 100)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 100)               0         
_____________________________________________________

In [54]:
y_pred = model.predict_classes(test_xx)
y_pred

array([0, 6, 3, 3, 6, 1, 1, 6, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 3, 6, 3, 3, 3, 6, 6, 3, 6, 6,
       6, 3, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 5, 3, 3, 6, 6, 6, 0, 6,
       6], dtype=int64)

In [56]:
model.evaluate(test_xx,dummy_class_test)

70/70 [==============================] - 0s 6ms/step


[13.272114310945783, 0.17142856972558157]